<a href="https://colab.research.google.com/github/Sreyas2403/StockAnalysis_Patterns-Indicators/blob/main/10csvfiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nsepython

In [ ]:
import nsepython as nse
import pandas as pd

def equity_history(symbol, series, start_date, end_date):
    # Fetch equity history for the selected symbol
    return nse.equity_history(symbol, series, start_date, end_date)

def calculate_rsi(data, window):
    delta = data['CH_CLOSING_PRICE'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Define the stock symbol and date range
symbol = input("Enter the symbol: ")
series = "EQ"
start_date = input("Enter the start date (yyyy-mm-dd): ")
end_date = input("Enter the end date (yyyy-mm-dd): ")

# Fetch equity data
equity_data = equity_history(symbol, series, start_date, end_date)
stock_df = pd.DataFrame(equity_data)
stock_df['mTIMESTAMP'] = pd.to_datetime(stock_df['mTIMESTAMP'])
stock_df = stock_df.sort_values(by='mTIMESTAMP')

# Convert necessary columns to numeric
stock_df['CH_CLOSING_PRICE'] = pd.to_numeric(stock_df['CH_CLOSING_PRICE'], errors='coerce')

# Drop rows with NaN values in key columns after conversion
stock_df = stock_df.dropna(subset=['CH_CLOSING_PRICE'])

# Calculate RSI
window_length = 14
stock_df['RSI'] = calculate_rsi(stock_df, window_length)

# Identify RSI values in valleys and peaks
prospect_list_valleys = []
prospect_list_peaks = []

for i in range(window_length, len(stock_df) - window_length):
    rsi_value = stock_df['RSI'].iloc[i]
    left_values = stock_df['RSI'].iloc[i-window_length:i]
    right_values = stock_df['RSI'].iloc[i+1:i+1+window_length]

    if all(abs(rsi_value - lv) > 1.5 for lv in left_values) and all(abs(rsi_value - rv) > 1.5 for rv in right_values):
        prospect_list_valleys.append(rsi_value)

    if all(abs(rsi_value - lv) > 1.5 for lv in left_values) and all(abs(rsi_value - rv) > 1.5 for rv in right_values):
        prospect_list_peaks.append(rsi_value)

# Print the prospect lists
print("Prospect List in Valleys (RSI values):")
print(prospect_list_valleys)

print("\nProspect List in Peaks (RSI values):")
print(prospect_list_peaks)


Enter the symbol: SBIN
Enter the start date (yyyy-mm-dd): 01-01-2021
Enter the end date (yyyy-mm-dd): 31-12-2023
Prospect List in Valleys (RSI values):
[51.73542894564504, 49.48979591836734, 44.15422885572137, 81.03225806451611, 79.19921874999997, 88.89294848374129, 95.0892857142857, 70.926143024619, 44.55825864276569, 23.743922204213817, 56.486622917718314, 78.74080130825843, 77.21297107800181, 68.72074882995322, 66.71974522292994, 95.41284403669725, 78.91891891891895, 73.178391959799, 71.05084745762714, 66.05723370429254, 46.45328719723181, 39.152371342078744, 37.415621986499545, 40.873015873015866, 44.454799627213404, 42.760942760942775, 60.07827788649706, 32.5693606755127, 22.42547425474261, 25.778816199376976, 30.569514237855984, 42.09748892171346, 72.64705882352945, 95.76490924805542, 46.72379032258065, 52.10409745293464, 41.62265962554011, 30.011587485515662, 26.829268292682897, 23.345123799898914, 54.04089581304772, 56.0271646859083, 29.29542645241041, 42.4160346695558, 76.8987

In [ ]:
import pandas as pd
import nsepython as nse
import numpy as np

# Define the stock symbol and series
symbol = input("Enter the symbol: ")
series = "EQ"
start_date = input("Enter the start date (dd-mm-yyyy): ")
end_date = input("Enter the end date (dd-mm-yyyy): ")

# Fetch equity history for the selected symbol
equity_data = nse.equity_history(symbol, series, start_date, end_date)
stock_df = pd.DataFrame(equity_data)
stock_df['mTIMESTAMP'] = pd.to_datetime(stock_df['mTIMESTAMP'])
stock_df = stock_df.sort_values(by='mTIMESTAMP')

# Convert necessary columns to numeric
stock_df['CH_CLOSING_PRICE'] = pd.to_numeric(stock_df['CH_CLOSING_PRICE'], errors='coerce')

# Drop rows with NaN values in key columns after conversion
stock_df = stock_df.dropna(subset=['CH_CLOSING_PRICE'])

# Calculate the RSI (Relative Strength Index)
def calculate_rsi(data, window=14):
    delta = data['CH_CLOSING_PRICE'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

stock_df['RSI'] = calculate_rsi(stock_df)

# Identify RSI prospect lists for valleys and peaks
prospect_list_valleys = []
prospect_list_peaks = []

# Iterate over the RSI values
for i in range(10, len(stock_df) - 10):
    rsi_value = stock_df['RSI'].iloc[i]
    left_rsi_values = stock_df['RSI'].iloc[i-10:i]
    right_rsi_values = stock_df['RSI'].iloc[i+1:i+11]

    # Check for valleys
    if all(left_rsi_values > rsi_value + 1.5) and all(right_rsi_values > rsi_value + 1.5):
        prospect_list_valleys.append(rsi_value)

    # Check for peaks
    if all(left_rsi_values < rsi_value - 1.5) and all(right_rsi_values < rsi_value - 1.5):
        prospect_list_peaks.append(rsi_value)

# Print the prospect lists
print("RSI Prospect List in Valleys:")
print(prospect_list_valleys)

print("RSI Prospect List in Peaks:")
print(prospect_list_peaks)


Enter the symbol: SBIN
Enter the start date (dd-mm-yyyy): 01-01-2021
Enter the end date (dd-mm-yyyy): 31-12-2023
RSI Prospect List in Valleys:
[23.743922204213817, 42.760942760942775, 22.42547425474261, 46.72379032258065, 11.902614968439991, 22.631578947368425, 28.41260037059918, 18.409090909090978, 47.629796839729124, 17.465069860279442, 24.064171122994665]
RSI Prospect List in Peaks:
[95.0892857142857, 95.41284403669725, 95.76490924805542, 56.0271646859083, 80.75268817204307, 80.00000000000003, 67.34693877551018, 86.90884167237833, 63.43187660668374, 83.59173126614984, 74.67057101024895, 87.45910577971665]


In [ ]:
prospect_list_valleys_df = pd.DataFrame(prospect_list_valleys)
prospect_list_peaks_df = pd.DataFrame(prospect_list_peaks)

#Saving to excel files
prospect_list_valleys_df.to_excel('prospect_rsi_valleys.xlsx', index = True )
prospect_list_peaks_df.to_excel('prospect_rsi_peaks.xlsx', index = False)

In [ ]:
import pandas as pd
import os

# Define the directory containing the CSV files
csv_directory = '/content/Data'  # Update this path as your file

# List of CSV files for different sectors and industries
sector_industry_files = ['/content/Data/ind_nifty_privatebanklist.csv', '/content/Data/ind_niftyautolist.csv', '/content/Data/ind_niftybanklist.csv',
'/content/Data/ind_niftyconsumerdurableslist.csv','/content/Data/ind_niftyfinancelist.csv', '/content/Data/ind_niftyfmcglist.csv',
'/content/Data/ind_niftyhealthcarelist.csv','/content/Data/ind_niftyitlist.csv' , '/content/Data/ind_niftymedialist.csv',
'/content/Data/ind_niftymetallist.csv', '/content/Data/ind_niftyoilgaslist.csv', '/content/Data/ind_niftypharmalist.csv',
'/content/Data/ind_niftypsubanklist.csv', '/content/Data/ind_niftyrealtylist.csv' ]



nifty500_file = '/content/ind_nifty500list.csv'
medium_cap_file = '/content/ind_niftymidcap150list.csv'
small_cap_file = '/content/ind_niftysmallcap250list.csv'

# Read Nifty 500 CSV file
nifty500_df = pd.read_csv(os.path.join(csv_directory, nifty500_file))
nifty500_symbols = nifty500_df['Symbol'].tolist()

# Read Cap CSV files
medium_cap_df = pd.read_csv(os.path.join(csv_directory, medium_cap_file))
small_cap_df = pd.read_csv(os.path.join(csv_directory, small_cap_file))

# Function to determine cap
def determine_cap(symbol):
    if symbol in medium_cap_df['Symbol'].tolist():
        return 'Medium'
    elif symbol in small_cap_df['Symbol'].tolist():
        return 'Small'
    else:
        return 'None'

# Consolidate data
consolidated_data = []

for file in sector_industry_files:
    sector_industry_df = pd.read_csv(os.path.join(csv_directory, file))
    industry = sector_industry_df['Industry'][0]  # Assuming the industry is the same for all rows in this file

    for symbol in sector_industry_df['Symbol']:
        nifty500_status = 'Nifty 500' if symbol in nifty500_symbols else 'None'
        cap = determine_cap(symbol)

        consolidated_data.append({
            'Symbol': symbol,
            'Nifty 500': nifty500_status,
            'Industry': industry,
            'Cap': cap
        })

# Create a DataFrame from the consolidated data
consolidated_df = pd.DataFrame(consolidated_data)

# Save the consolidated DataFrame to an Excel file
output_file = 'newconsolidated_data.xlsx'
consolidated_df.to_excel(output_file, index=False)

print("Consolidated data has been saved to", output_file)


Consolidated data has been saved to newconsolidated_data.xlsx


In [ ]:
import pandas as pd

# List of CSV files for different sectors and industries
sector_industry_files = ['/content/Data/ind_nifty_privatebanklist.csv', '/content/Data/ind_niftyautolist.csv', '/content/Data/ind_niftybanklist.csv',
'/content/Data/ind_niftyconsumerdurableslist.csv','/content/Data/ind_niftyfinancelist.csv', '/content/Data/ind_niftyfmcglist.csv',
'/content/Data/ind_niftyhealthcarelist.csv','/content/Data/ind_niftyitlist.csv' , '/content/Data/ind_niftymedialist.csv',
'/content/Data/ind_niftymetallist.csv', '/content/Data/ind_niftyoilgaslist.csv', '/content/Data/ind_niftypharmalist.csv',
'/content/Data/ind_niftypsubanklist.csv', '/content/Data/ind_niftyrealtylist.csv' ]

nifty500_file = '/content/ind_nifty500list.csv'
medium_cap_file = '/content/ind_niftymidcap150list.csv'
small_cap_file = '/content/ind_niftysmallcap250list.csv'

# Read Nifty 500 CSV file
nifty500_df = pd.read_csv(nifty500_file)
nifty500_symbols = nifty500_df['Symbol'].tolist()

# Read Cap CSV files
medium_cap_df = pd.read_csv(medium_cap_file)
small_cap_df = pd.read_csv(small_cap_file)

# Function to determine cap
def determine_cap(symbol):
    if symbol in medium_cap_df['Symbol'].tolist():
        return 'Medium'
    elif symbol in small_cap_df['Symbol'].tolist():
        return 'Small'
    else:
        return 'None'

# Consolidate data
consolidated_data = []

for file in sector_industry_files:
    sector_industry_df = pd.read_csv(file)
    industry = sector_industry_df['Industry'][0]  # Assuming the industry is the same for all rows in this file

    for symbol in sector_industry_df['Symbol']:
        nifty500_status = 'Nifty 500' if symbol in nifty500_symbols else 'None'
        cap = determine_cap(symbol)

        consolidated_data.append({
            'Symbol': symbol,
            'Nifty 500': nifty500_status,
            'Industry': industry,
            'Cap': cap
        })

# Create a DataFrame from the consolidated data
consolidated_df = pd.DataFrame(consolidated_data)

# Save the consolidated DataFrame to an Excel file
output_file = 'consolidated_data.xlsx'
consolidated_df.to_excel(output_file, index=False)

print("Consolidated data has been saved to", output_file)


Consolidated data has been saved to consolidated_data.xlsx


In [ ]:
import pandas as pd
import os

# Define the directory containing the CSV files
csv_directory = '/content/Data'  # Update this path as your file

# List of CSV files for different sectors and industries
sector_industry_files = [
    'ind_nifty_privatebanklist.csv', 'ind_niftyautolist.csv', 'ind_niftybanklist.csv',
    'ind_niftyconsumerdurableslist.csv', 'ind_niftyfinancelist.csv', 'ind_niftyfmcglist.csv',
    'ind_niftyhealthcarelist.csv', 'ind_niftyitlist.csv', 'ind_niftymedialist.csv',
    'ind_niftymetallist.csv', 'ind_niftyoilgaslist.csv', 'ind_niftypharmalist.csv',
    'ind_niftypsubanklist.csv', 'ind_niftyrealtylist.csv'
]


# Directory containing theme sector files
theme_sector_directory = '/content/ThemeData'  # Update this path with your actual theme sector directory

nifty50_file = '/content/ind_nifty50list.csv'
nifty100_file = '/content/ind_nifty100list.csv'
nifty500_file = '/content/ind_nifty500list.csv'
medium_cap_file = '/content/ind_niftymidcap150list.csv'
small_cap_file = '/content/ind_niftysmallcap250list.csv'


# Read Nifty 500 CSV file
nifty500_df = pd.read_csv(os.path.join(csv_directory, nifty500_file))
nifty500_symbols = nifty500_df['Symbol'].tolist()

# Read Cap CSV files
medium_cap_df = pd.read_csv(os.path.join(csv_directory, medium_cap_file))
small_cap_df = pd.read_csv(os.path.join(csv_directory, small_cap_file))

# Read Nifty50 and Nifty100 CSV files
nifty50_df = pd.read_csv(os.path.join(csv_directory, nifty50_file))
nifty100_df = pd.read_csv(os.path.join(csv_directory, nifty100_file))
nifty50_symbols = nifty50_df['Symbol'].tolist()
nifty100_symbols = nifty100_df['Symbol'].tolist()

# Read theme sector files
theme_sector_data = {}
for file in os.listdir(theme_sector_directory):
    if file.endswith('.csv'):
        theme_df = pd.read_csv(os.path.join(theme_sector_directory, file))
        theme_sector = os.path.splitext(file)[0]  # Get the theme sector name from the file name
        for _, row in theme_df.iterrows():
            symbol = row['Symbol']
            industry = row['Industry']
            theme_sector_data[symbol] = (industry, theme_sector)

# Function to determine cap
def determine_cap(symbol):
    if symbol in medium_cap_df['Symbol'].tolist():
        return 'Medium'
    elif symbol in small_cap_df['Symbol'].tolist():
        return 'Small'
    else:
        return 'None'

# Consolidate data
consolidated_data = []

for file in sector_industry_files:
    sector_industry_df = pd.read_csv(os.path.join(csv_directory, file))
    industry = sector_industry_df['Industry'][0]  # Assuming the industry is the same for all rows in this file
    sector = os.path.splitext(file)[0]  # Get the sector name from the file name

    for symbol in sector_industry_df['Symbol']:
        nifty500_status = 'True' if symbol in nifty500_symbols else 'False'
        cap = determine_cap(symbol)
        nifty50_status = 'True' if symbol in nifty50_symbols else 'False'
        nifty100_status = 'True' if symbol in nifty100_symbols else 'False'

        theme_industry, theme_sector = theme_sector_data.get(symbol, ('None', 'None'))

        consolidated_data.append({
            'Symbol': symbol,
            'Nifty50': nifty50_status,
            'Nifty100': nifty100_status,
            'Nifty 500': nifty500_status,
            'Industry': industry,
            'Cap': cap,
            'Sector': sector,
            'Theme_Industry': theme_industry,
            'Theme_Sector': theme_sector
        })

# Create a DataFrame from the consolidated data
consolidated_df = pd.DataFrame(consolidated_data)

# Save the consolidated DataFrame to an Excel file
output_file = 'finalconsolidated_data.xlsx'
consolidated_df.to_excel(output_file, index=False)

print("Consolidated data has been saved to", output_file)


Consolidated data has been saved to finalconsolidated_data.xlsx


In [ ]:
#1967 code2
import pandas as pd
import os

# Define the directory containing the CSV files
csv_directory = '/content/Industry'  # Update this path as needed

# List of CSV files for different sectors and industries
sector_industry_files = [
    'ind_nifty_privatebanklist.csv', 'ind_niftyautolist.csv', 'ind_niftybanklist.csv',
    'ind_niftyconsumerdurableslist.csv', 'ind_niftyfinancelist.csv', 'ind_niftyfmcglist.csv',
    'ind_niftyhealthcarelist.csv', 'ind_niftyitlist.csv', 'ind_niftymedialist.csv',
    'ind_niftymetallist.csv', 'ind_niftyoilgaslist.csv', 'ind_niftypharmalist.csv',
    'ind_niftypsubanklist.csv', 'ind_niftyrealtylist.csv'
]

# List of CSV files for theme sectors
# List of CSV files for theme sectors
theme_sector_directory = '/content/Theme'

# Read NSE CSV file
nse_file = '/content/NSE_stocks.csv'
nse_df = pd.read_csv(os.path.join(csv_directory, nse_file))
nse_symbols = nse_df['SYMBOL'].tolist()

# Read Cap CSV files
medium_cap_file = '/content/ind_niftymidcap150list.csv'
small_cap_file = '/content/ind_niftysmallcap250list.csv'
medium_cap_df = pd.read_csv(os.path.join(csv_directory, medium_cap_file))
small_cap_df = pd.read_csv(os.path.join(csv_directory, small_cap_file))

# Read Nifty50 and Nifty100 CSV files
nifty50_file = '/content/ind_nifty50list.csv'
nifty100_file = '/content/ind_nifty100list.csv'
nifty500_file = '/content/ind_nifty500list.csv'
nifty50_df = pd.read_csv(os.path.join(csv_directory, nifty50_file))
nifty100_df = pd.read_csv(os.path.join(csv_directory, nifty100_file))
nifty500_df = pd.read_csv(os.path.join(csv_directory, nifty500_file))
nifty50_symbols = nifty50_df['Symbol'].tolist()
nifty100_symbols = nifty100_df['Symbol'].tolist()
nifty500_symbols = nifty500_df['Symbol'].tolist()

# Read theme sector files
theme_sector_data = {}
for file in os.listdir(theme_sector_directory):
    if file.endswith('.csv'):
        theme_df = pd.read_csv(os.path.join(theme_sector_directory, file))
        theme_sector = os.path.splitext(file)[0]  # Get the theme sector name from the file name
        for _, row in theme_df.iterrows():
            symbol = row['Symbol']
            industry = row['Industry']
            theme_sector_data[symbol] = (industry, theme_sector)

# Function to determine cap
def determine_cap(symbol):
    if symbol in medium_cap_df['Symbol'].tolist():
        return 'Medium'
    elif symbol in small_cap_df['Symbol'].tolist():
        return 'Small'
    else:
        return 'None'

# Consolidate data
consolidated_data = []

for symbol in nse_symbols:
    nifty500_status = 'Nifty 500' if symbol in nifty500_symbols else 'None'
    cap = determine_cap(symbol)
    nifty50_status = 'True' if symbol in nifty50_symbols else 'False'
    nifty100_status = 'True' if symbol in nifty100_symbols else 'False'

    industry, sector = 'None', 'None'
    for file in sector_industry_files:
        sector_industry_df = pd.read_csv(os.path.join(csv_directory, file))
        if symbol in sector_industry_df['Symbol'].tolist():
            industry = sector_industry_df.loc[sector_industry_df['Symbol'] == symbol, 'Industry'].values[0]
            sector = os.path.splitext(file)[0]
            break

    theme_industry, theme_sector = theme_sector_data.get(symbol, ('None', 'None'))

    consolidated_data.append({
        'Symbol': symbol,
        'Nifty 500': nifty500_status,
        'Industry': industry,
        'Cap': cap,
        'Sector': sector,
        'Nifty50': nifty50_status,
        'Nifty100': nifty100_status,
        'Theme_Industry': theme_industry,
        'Theme_Sector': theme_sector
    })

# Create a DataFrame from the consolidated data
consolidated_df = pd.DataFrame(consolidated_data)

# Save the consolidated DataFrame to an Excel file
output_file = '1967symbols_data.xlsx'
consolidated_df.to_excel(output_file, index=False)

print("Consolidated data has been saved to", output_file)


Consolidated data has been saved to 1967symbols_data.xlsx


In [ ]:
import pandas as pd
import os

# Define the directory containing the CSV files
csv_directory = '/content/Data'  # Update this path as your file

# List of CSV files for different sectors and industries
sector_industry_files = [
    'ind_nifty_privatebanklist.csv', 'ind_niftyautolist.csv', 'ind_niftybanklist.csv',
    'ind_niftyconsumerdurableslist.csv', 'ind_niftyfinancelist.csv', 'ind_niftyfmcglist.csv',
    'ind_niftyhealthcarelist.csv', 'ind_niftyitlist.csv', 'ind_niftymedialist.csv',
    'ind_niftymetallist.csv', 'ind_niftyoilgaslist.csv', 'ind_niftypharmalist.csv',
    'ind_niftypsubanklist.csv', 'ind_niftyrealtylist.csv'
]

# Directory containing theme sector files
theme_sector_directory = '/content/ThemeData'  # Update this path with your actual theme sector directory

nifty50_file = '/content/ind_nifty50list.csv'
nifty100_file = '/content/ind_nifty100list.csv'
nifty500_file = '/content/ind_nifty500list.csv'
medium_cap_file = '/content/ind_niftymidcap150list.csv'
small_cap_file = '/content/ind_niftysmallcap250list.csv'


# Read Nifty 500 CSV file
nifty500_df = pd.read_csv(os.path.join(csv_directory, nifty500_file))
nifty500_symbols = nifty500_df['Symbol'].tolist()

# Read Cap CSV files
medium_cap_df = pd.read_csv(os.path.join(csv_directory, medium_cap_file))
small_cap_df = pd.read_csv(os.path.join(csv_directory, small_cap_file))

# Read Nifty50 and Nifty100 CSV files
nifty50_df = pd.read_csv(os.path.join(csv_directory, nifty50_file))
nifty100_df = pd.read_csv(os.path.join(csv_directory, nifty100_file))
nifty50_symbols = nifty50_df['Symbol'].tolist()
nifty100_symbols = nifty100_df['Symbol'].tolist()

# Read theme sector files
theme_sector_data = {}
for file in os.listdir(theme_sector_directory):
    if file.endswith('.csv'):
        theme_df = pd.read_csv(os.path.join(theme_sector_directory, file))
        theme_sector = os.path.splitext(file)[0]  # Get the theme sector name from the file name
        for _, row in theme_df.iterrows():
            symbol = row['Symbol']
            industry = row['Industry']
            theme_sector_data[symbol] = (industry, theme_sector)

# Function to determine cap
def determine_cap(symbol):
    if symbol in medium_cap_df['Symbol'].tolist():
        return 'Medium'
    elif symbol in small_cap_df['Symbol'].tolist():
        return 'Small'
    else:
        return 'None'

# Function to calculate RSI
def calculate_rsi(prices, window=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to find peaks and valleys
def find_peaks_and_valleys(rsi_series):
    peaks = []
    valleys = []
    for i in range(1, len(rsi_series) - 1):
        if rsi_series[i] > rsi_series[i - 1] and rsi_series[i] > rsi_series[i + 1]:
            peaks.append(rsi_series[i])
        elif rsi_series[i] < rsi_series[i - 1] and rsi_series[i] < rsi_series[i + 1]:
            valleys.append(rsi_series[i])
    return peaks, valleys

# Consolidate data
consolidated_data = []

for file in sector_industry_files:
    sector_industry_df = pd.read_csv(os.path.join(csv_directory, file))
    industry = sector_industry_df['Industry'][0]  # Assuming the industry is the same for all rows in this file
    sector = os.path.splitext(file)[0]  # Get the sector name from the file name

    for symbol in sector_industry_df['Symbol']:
        nifty500_status = 'Nifty 500' if symbol in nifty500_symbols else 'None'
        cap = determine_cap(symbol)
        nifty50_status = 'True' if symbol in nifty50_symbols else 'False'
        nifty100_status = 'True' if symbol in nifty100_symbols else 'False'

        theme_industry, theme_sector = theme_sector_data.get(symbol, ('None', 'None'))

        # Assuming we have a DataFrame `rsi_data` with 'Symbol' and 'Close' prices for calculating RSI
        rsi_data = pd.DataFrame({
            'Symbol': [symbol] * 100,  # Replace with your actual data source
            'CH_CLOSING_PRICE': pd.Series(range(100))  # Replace with actual closing prices
        })

        if symbol in rsi_data['Symbol'].unique():
            symbol_rsi_data = rsi_data[rsi_data['Symbol'] == symbol]
            symbol_rsi_data['RSI'] = calculate_rsi(symbol_rsi_data['CH_CLOSING_PRICE'])
            peaks, valleys = find_peaks_and_valleys(symbol_rsi_data['RSI'])
        else:
            peaks, valleys = [], []

        consolidated_data.append({
            'Symbol': symbol,
            'Nifty 500': nifty500_status,
            'Industry': industry,
            'Cap': cap,
            'Sector': sector,
            'Nifty50': nifty50_status,
            'Nifty100': nifty100_status,
            'Theme_Industry': theme_industry,
            'Theme_Sector': theme_sector,
            'RSI_Peak': peaks,
            'RSI_Valleys': valleys
        })

# Create a DataFrame from the consolidated data
consolidated_df = pd.DataFrame(consolidated_data)

# Save the consolidated DataFrame to an Excel file
output_file = 'rsi_consolidated_data.xlsx'
consolidated_df.to_excel(output_file, index=False)

print("Consolidated data has been saved to", output_file)


Consolidated data has been saved to rsi_consolidated_data.xlsx


In [ ]:
import pandas as pd
import os
import yfinance as yf

# Define the directory containing the CSV files
csv_directory = '/content/Data'  # Update this path as your file

# List of CSV files for different sectors and industries
sector_industry_files = [
    'ind_nifty_privatebanklist.csv', 'ind_niftyautolist.csv', 'ind_niftybanklist.csv',
    'ind_niftyconsumerdurableslist.csv', 'ind_niftyfinancelist.csv', 'ind_niftyfmcglist.csv',
    'ind_niftyhealthcarelist.csv', 'ind_niftyitlist.csv', 'ind_niftymedialist.csv',
    'ind_niftymetallist.csv', 'ind_niftyoilgaslist.csv', 'ind_niftypharmalist.csv',
    'ind_niftypsubanklist.csv', 'ind_niftyrealtylist.csv'
]

# Directory containing theme sector files
theme_sector_directory = '/content/ThemeData'  # Update this path with your actual theme sector directory

nifty500_file = '/content/ind_nifty500list.csv'
medium_cap_file = '/content/ind_niftymidcap150list.csv'
small_cap_file = '/content/ind_niftymidcap150list.csv'
nifty50_file = '/content/ind_nifty50list.csv'
nifty100_file = '/content/ind_nifty100list.csv'

# Read Nifty 500 CSV file
nifty500_df = pd.read_csv(os.path.join(csv_directory, nifty500_file))
nifty500_symbols = nifty500_df['Symbol'].tolist()

# Read Cap CSV files
medium_cap_df = pd.read_csv(os.path.join(csv_directory, medium_cap_file))
small_cap_df = pd.read_csv(os.path.join(csv_directory, small_cap_file))

# Read Nifty50 and Nifty100 CSV files
nifty50_df = pd.read_csv(os.path.join(csv_directory, nifty50_file))
nifty100_df = pd.read_csv(os.path.join(csv_directory, nifty100_file))
nifty50_symbols = nifty50_df['Symbol'].tolist()
nifty100_symbols = nifty100_df['Symbol'].tolist()

# Read theme sector files
theme_sector_data = {}
for file in os.listdir(theme_sector_directory):
    if file.endswith('.csv'):
        theme_df = pd.read_csv(os.path.join(theme_sector_directory, file))
        theme_sector = os.path.splitext(file)[0]  # Get the theme sector name from the file name
        for _, row in theme_df.iterrows():
            symbol = row['Symbol']
            industry = row['Industry']
            theme_sector_data[symbol] = (industry, theme_sector)

# Function to determine cap
def determine_cap(symbol):
    if symbol in medium_cap_df['Symbol'].tolist():
        return 'Medium'
    elif symbol in small_cap_df['Symbol'].tolist():
        return 'Small'
    else:
        return 'None'

# Function to calculate RSI
def calculate_rsi(prices, window=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to find peaks and valleys
def find_peaks_and_valleys(rsi_series):
    peaks = []
    valleys = []
    for i in range(1, len(rsi_series) - 1):
        if rsi_series[i] > rsi_series[i - 1] and rsi_series[i] > rsi_series[i + 1]:
            peaks.append(rsi_series[i])
        elif rsi_series[i] < rsi_series[i - 1] and rsi_series[i] < rsi_series[i + 1]:
            valleys.append(rsi_series[i])
    return peaks, valleys

# Function to fetch historical data and calculate RSI for a symbol
def get_rsi_for_symbol(symbol, period='1y', interval='1d'):
    try:
        symbol_yf = symbol + '.NS'
        data = yf.download(symbol_yf, period=period, interval=interval)
        data['RSI'] = calculate_rsi(data['Close'])
        return data['RSI']
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return pd.Series()

# Consolidate data
consolidated_data = []

for file in sector_industry_files:
    sector_industry_df = pd.read_csv(os.path.join(csv_directory, file))
    industry = sector_industry_df['Industry'][0]  # Assuming the industry is the same for all rows in this file
    sector = os.path.splitext(file)[0]  # Get the sector name from the file name

    for symbol in sector_industry_df['Symbol']:
        nifty500_status = 'Nifty 500' if symbol in nifty500_symbols else 'None'
        cap = determine_cap(symbol)
        nifty50_status = 'True' if symbol in nifty50_symbols else 'False'
        nifty100_status = 'True' if symbol in nifty100_symbols else 'False'

        theme_industry, theme_sector = theme_sector_data.get(symbol, ('None', 'None'))

        rsi_series = get_rsi_for_symbol(symbol)
        peaks, valleys = find_peaks_and_valleys(rsi_series)

        consolidated_data.append({
            'Symbol': symbol,
            'Nifty50': nifty50_status,
            'Nifty100': nifty100_status,
            'Nifty 500': nifty500_status,
            'Industry': industry,
            'Cap': cap,
            'Sector': sector,
            'Theme_Industry': theme_industry,
            'Theme_Sector': theme_sector,
            'RSI_Peak': peaks,
            'RSI_Valleys': valleys
        })

# Create a DataFrame from the consolidated data
consolidated_df = pd.DataFrame(consolidated_data)

# Save the consolidated DataFrame to an Excel file
output_file = 'rsi_consolidated_data.xlsx'
consolidated_df.to_excel(output_file, index=False)

print("Consolidated data has been saved to", output_file)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Consolidated data has been saved to rsi_consolidated_data.xlsx


In [ ]:
!pip install ta


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=3fd23e2a74ba7b1106b2b9d84adb3ee7febf9b293efc85994e6357462d861293
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
!pip install yfinance